In [1]:
import re
import nltk
import string
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

# Remove punctuation from a list of words
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

# Remove stopwords from a list of words
def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

# Tokenize the input sentence and also lemmatize its words
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

def get_signature(sense):
    signature = []
    for word in tokenize_sentence(sense.definition()):  # definition tokenization
        signature.append(word)
    for example in sense.examples():  # example tokenization
        for word in tokenize_sentence(example):
            # Merge definition and examples
            signature.append(word)
    return signature

In [3]:
def read_corpus(txt_file):
    
    with open(txt_file, encoding='utf-8') as file:
        # for each doc create list of pre-processed words in that doc (list of lists)
        documents_words = []

        for line in file:
            if "<doc" in line:  # tag for new doc
                document_words = []  # list of words that will be part of the document
                while True:
                    next_line = file.readline()  # read next line

                    # remove unuseful tags
                    next_line_proc = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                    if "</doc>" in next_line:
                        break
                    # pre-processing steps
                    sentence_words = pre_processing(next_line_proc)
                    document_words.extend(sentence_words)
                documents_words.append(document_words)
        file.close()

    print("Documents number: ", len(documents_words))
    
    return documents_words

In [4]:
# Topic modelling using gensim
from gensim import corpora, models

topic_num = 10
topic_words_num = 5

def topic_modelling(documents_words):
    
    
    # Create a dict with integer keys for all words
    dictionary_LDA = corpora.Dictionary(documents_words)

    # delete all terms that do NOT appear in at least 3 documents.
    #delete all terms that appear in more than 50% of documents (see filter_extremes official doc).
    dictionary_LDA.filter_extremes(no_below=3)

    # Converts each document into a list of BoW (list of (id_term, term_frequency) for each term in doc)
    corpus_idbow_freq = [dictionary_LDA.doc2bow(document_words) for document_words in documents_words]
    

    lda_model = models.LdaModel(corpus_idbow_freq, num_topics=topic_num, \
                                id2word=dictionary_LDA, \
                                passes=4, alpha=[0.01] * topic_num, \
                                eta=[0.01] * len(dictionary_LDA.keys()))
    
    return lda_model, corpus_idbow_freq


Show topics.

In [5]:
documents_words = read_corpus("travelling.txt")

model, corpus_idbow_freq = topic_modelling(documents_words)

topics = {'Topic_' + str(i): [(token, round(score, 3)) for token, score in model.show_topic(i, topn=topic_words_num)] for i in range(0, model.num_topics)}
for key, value in topics.items():
    print(topics[key])

Documents number:  100
[('lesson', 0.133), ('website', 0.034), ('pronunciation', 0.022), ('life', 0.019), ('activity', 0.018)]
[('__________________', 0.02), ('write', 0.017), ('rule', 0.016), ('word', 0.016), ('language', 0.015)]
[('happen', 0.018), ('conditionals', 0.018), ('result', 0.017), ('condition', 0.015), ('third', 0.014)]
[('journey', 0.023), ('rain', 0.016), ('trip', 0.015), ('tour', 0.015), ('place', 0.011)]
[('flight', 0.021), ('journey', 0.019), ('subject', 0.015), ('film', 0.013), ('trip', 0.013)]
[('word', 0.034), ('language', 0.018), ('article', 0.017), ('noun', 0.014), ('thing', 0.009)]
[('student', 0.038), ('lesson', 0.029), ('worksheet', 0.025), ('language', 0.018), ('online', 0.016)]
[('hotel', 0.029), ('book', 0.026), ('holiday', 0.025), ('beach', 0.018), ('money', 0.012)]
[('students', 0.063), ('exam', 0.032), ('sb', 0.025), ('book', 0.025), ('speak', 0.024)]
[('lesson', 0.023), ('student', 0.015), ('audio', 0.015), ('dailystep', 0.011), ('phrasal', 0.009)]


Show topics for each document.

In [10]:
print ("Documents topic list")
for i in range (0, len(corpus_idbow_freq)):
    print ("Doc", i, model[corpus_idbow_freq[i]])

Documents topic list
Doc 0 [(6, 0.5333893), (7, 0.46527943)]
Doc 1 [(7, 0.80343425), (8, 0.19437215)]
Doc 2 [(9, 0.9975737)]
Doc 3 [(5, 0.6709842), (7, 0.11149991), (8, 0.2170993)]
Doc 4 [(9, 0.99980974)]
Doc 5 [(5, 0.99993277)]
Doc 6 [(0, 0.9989784)]
Doc 7 [(1, 0.6054322), (2, 0.20323488), (6, 0.19078636)]
Doc 8 [(7, 0.9999108)]
Doc 9 [(2, 0.3870464), (3, 0.02362862), (5, 0.36366206), (7, 0.22543569)]
Doc 10 [(2, 0.9998624)]
Doc 11 [(2, 0.99970204)]
Doc 12 [(6, 0.96266425), (7, 0.036494467)]
Doc 13 [(0, 0.13591199), (6, 0.20773683), (8, 0.6560809)]
Doc 14 [(0, 0.15984409), (4, 0.030539991), (5, 0.6071806), (8, 0.2020906)]
Doc 15 [(1, 0.09429425), (2, 0.12214975), (5, 0.072534434), (8, 0.06488724), (9, 0.64604867)]
Doc 16 [(1, 0.077662215), (2, 0.92165446)]
Doc 17 [(2, 0.18730727), (7, 0.3487466), (9, 0.46373007)]
Doc 18 [(2, 0.999622)]
Doc 19 [(2, 0.7645136), (4, 0.23514019)]
Doc 20 [(2, 0.99981016)]
Doc 21 [(2, 0.9995929)]
Doc 22 [(2, 0.99958545)]
Doc 23 [(2, 0.99964994)]
Doc 24 [(1,